Install fluidsynth
```bash
sudo apt-get install fluidsynth
pip install pyfluidsynth
```

In [1]:
from typing import List
from pathlib import Path
import midifile_binding as mf 
import symusic as sm 
import pretty_midi as pm
import timeit
import os
import random
from time import sleep
from tqdm import tqdm
from random import sample
from collections import defaultdict

In [2]:
MIDI_DATASET_NAMES = ('maestro', 'musicnet', 'POP909')
DATASET_ROOT = "./symusic_benchmark_datasets"
SAMPLE_NUM = 10

def is_valid(f: str):
    try:
        sm.Score(f)
        return True
    except:
        return False

print("Scanning datasets to filter out invalid files...")
sleep(0.1)
random.seed(42)
MIDI_DATASET = {
    name: sample(sorted(list(filter(
        is_valid, 
        tqdm(list(map(str, Path(DATASET_ROOT).joinpath(name).rglob('*.mid*'))), desc=name)
    ))), SAMPLE_NUM) for name in MIDI_DATASET_NAMES
}
print(MIDI_DATASET)

Scanning datasets to filter out invalid files...


POP909: 100%|██████████| 2898/2898 [00:10<00:00, 280.17it/s]

{'maestro': ['symusic_benchmark_datasets/maestro/MIDI-Unprocessed_04_R1_2011_MID--AUDIO_R1-D2_06_Track06_wav.midi', 'symusic_benchmark_datasets/maestro/MIDI-UNPROCESSED_09-10_R1_2014_MID--AUDIO_09_R1_2014_wav--4.midi', 'symusic_benchmark_datasets/maestro/MIDI-Unprocessed_12_R1_2009_03-05_ORIG_MID--AUDIO_12_R1_2009_12_R1_2009_03_WAV.midi', 'symusic_benchmark_datasets/maestro/MIDI-Unprocessed_10_R1_2008_01-04_ORIG_MID--AUDIO_10_R1_2008_wav--4.midi', 'symusic_benchmark_datasets/maestro/MIDI-Unprocessed_08_R3_2008_01-05_ORIG_MID--AUDIO_08_R3_2008_wav--3.midi', 'symusic_benchmark_datasets/maestro/MIDI-Unprocessed_059_PIANO059_MID--AUDIO-split_07-07-17_Piano-e_2-03_wav--3.midi', 'symusic_benchmark_datasets/maestro/MIDI-Unprocessed_049_PIANO049_MID--AUDIO-split_07-06-17_Piano-e_2-06_wav--1.midi', 'symusic_benchmark_datasets/maestro/MIDI-Unprocessed_XP_15_R2_2004_01_ORIG_MID--AUDIO_15_R2_2004_02_Track02_wav.midi', 'symusic_benchmark_datasets/maestro/MIDI-Unprocessed_03_R3_2011_MID--AUDIO_R3-D1

In [3]:
sf3_path = "/home/lyk/Download/MuseScore_General.sf3"
sample_rate = 44100

synth_time = defaultdict(list)

for name, paths in MIDI_DATASET.items():
    print(f"Processing {name} dataset...")
    scores = [sm.Score(p) for p in paths]
    pms = [pm.PrettyMIDI(p) for p in paths]

    def render_score():
        synthesizer = sm.Synthesizer(sf3_path, sample_rate=sample_rate)
        for score in tqdm(scores, desc="Symusic rendering", leave=False):
            synthesizer.render(score, True)
    
    def render_pretty_midi():
        for pm in tqdm(pms, desc="PrettyMIDI rendering", leave=False):
            pm.fluidsynth(sample_rate, sf3_path)
    
    synth_time['symusic'].append(timeit.timeit(render_score, number=4) / 4)
    synth_time['fluidsynth'].append(timeit.timeit(render_pretty_midi, number=2) / 2)
    
print(synth_time)

Processing maestro dataset...


Processing musicnet dataset...


/home/lyk/miniforge3/envs/music/lib/python3.11/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


KeyboardInterrupt: 